In [4]:
# Pandas Dataframe Library
import json
import pandas as pd
# Machine learning libraries
import tensorflow as tf
import keras
import torch
#Library used for fine tuning
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
# HateBert Libarary
tokenizer = AutoTokenizer.from_pretrained("GroNLP/hateBERT")


def load_data():
    # Open train jsonl file
    with open('train.jsonl', 'r') as f:
        train_data = [json.loads(line) for line in f.readlines()]

    # Create a pandas DataFrame from the parsed json data
    train_df = pd.DataFrame(train_data)

    # Open validation jsonl file
    with open('val.jsonl', 'r') as f:
        val_data = [json.loads(line) for line in f.readlines()]

    # Create a pandas DataFrame from the parsed json data
    val_df = pd.DataFrame(val_data)

    # Open test jsonl file
    with open('test.jsonl', 'r') as f:
        test_data = [json.loads(line) for line in f.readlines()]

    # Create a pandas DataFrame from the parsed json data
    test_df = pd.DataFrame(test_data)


    return train_data, val_data, test_data


SyntaxError: trailing comma not allowed without surrounding parentheses (86588920.py, line 10)

In [5]:
def tokenize_data(data):
    tokenized_data = tokenizer(
        data["context"],
        data["target"],
        padding="max_length",
        max_length=20,
        truncation=True
    )
    tokenized_data["label"] = int(data["label"])
    return tokenized_data

def list_of_dicts_to_dict_of_lists(d):
    dic = d[0]
    keys = dic.keys()
    values = [dic.values() for dic in d]
    return {k: list(v) for k, v in zip(keys, zip(*values))}

def load_tokenized_data():
    train_data, val_data, test_data = load_data()
    tokenized_train = list_of_dicts_to_dict_of_lists([tokenize_data(data) for data in train_data])
    tokenized_val = list_of_dicts_to_dict_of_lists([tokenize_data(data) for data in val_data])
    tokenized_test = list_of_dicts_to_dict_of_lists([tokenize_data(data) for data in test_data])
    
    
    return tokenized_train, tokenized_val, tokenized_test






tokenized_train_data, tokenized_val_data, tokenized_test_data = load_tokenized_data()

# load pre-trained HateBert
model = AutoModelForMaskedLM.from_pretrained("GroNLP/hateBERT")


In [6]:


# Define the training and validation datasets using DataLoader


train_dataset = tf.data.Dataset.from_tensor_slices(tokenized_train_data)
eval_dataset = tf.data.Dataset.from_tensor_slices(tokenized_val_data)



# Define the training arguments for the Trainer
training_args = TrainingArguments(
    output_dir='./results',           # output directory
    num_train_epochs=5,               # total number of training epochs
    per_device_train_batch_size=20,   # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
)




trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

trainer.train()


NameError: name 'TrainingArguments' is not defined